# How to write error-resistant workflows

## Introduction
In this tutorial, we will show how to implement the error handling in a WorkGraph.


Load the AiiDA profile.

In [5]:
%load_ext aiida
from aiida import load_profile
load_profile()

Profile<uuid='57ccbf7d9e2b41b39edb2bfdaf725feb' name='default'>

## Normal WorkGraph
We will show how to implement the error handlers for the `ArithmeticAddCalculation`. We start by creating a normal WorkGraph:

In [8]:
from aiida_workgraph import WorkGraph
from aiida import orm
from aiida.calculations.arithmetic.add import ArithmeticAddCalculation


wg = WorkGraph("normal_graph")
wg.tasks.new(ArithmeticAddCalculation, name="add1")

#------------------------- Submit the calculation -------------------
code = orm.load_code("add@localhost")
wg.submit(inputs={"add1": {"code": code,
                            "x": orm.Int(1),
                           "y": orm.Int(2)
                           }},
          wait=True)
print("Task finished OK? ", wg.tasks["add1"].process.is_finished_ok)
print("Exit code: ", wg.tasks["add1"].process.exit_code)
print("Exit Message: ", wg.tasks["add1"].process.exit_message)


WorkGraph process created, PK: 77323
Task finished OK?  True
Exit code:  None
Exit Message:  None


## Error code

If the computed sum of the inputs x and y is negative, the `ArithmeticAddCalculation` fails with exit code 410. Let's reset the WorkGraph and modify the inputs:

In [9]:
wg.reset()
wg.submit(inputs={"add1": {"code": code,
                            "x": orm.Int(1),
                           "y": orm.Int(-2)
                           }},
          wait=True)
print("Task finished OK? ", wg.tasks["add1"].process.is_finished_ok)
print("Exit code: ", wg.tasks["add1"].process.exit_code)
print("Exit Message: ", wg.tasks["add1"].process.exit_message)

WorkGraph process created, PK: 77331
Task finished OK?  False
Exit code:  ExitCode(status=410, message='The sum of the operands is a negative number.', invalidates_cache=False)
Exit Message:  The sum of the operands is a negative number.


We can confirm that the task fails by:

In [10]:
%verdi process status 77331

WorkGraph<normal_graph><77331> Finished [302]
    └── ArithmeticAddCalculation<77332> Finished [410]


## Error handling

To “register” a error handler for a WorkGraph, you simply define a function that takes the `self` as its single argument and set it as the `error_hanlders` of the WorkGraph:

In [11]:
from aiida_workgraph import WorkGraph
from aiida import orm
from aiida.calculations.arithmetic.add import ArithmeticAddCalculation

def handle_negative_sum(self):
    """Handle negative sum by resetting the task and changing the inputs.
    self is the WorkGraph instance, thus we can access the tasks and the context.
    """
    node = self.get_task_state_info("add1", "process")
    if node and node.exit_code and node.exit_code.status == 410:
        self.report(f"Run error handler: handle_negative_sum.")
        self.reset_task("add1")
        # modify task inputs
        task = self.ctx.tasks["add1"]
        # the actual input values are stored in the properties dictionary
        task["properties"]["x"]["value"] = orm.Int(abs(-task["properties"]["x"]["value"]))
        task["properties"]["y"]["value"] = orm.Int(abs(-task["properties"]["y"]["value"]))

wg = WorkGraph("normal_graph")
wg.tasks.new(ArithmeticAddCalculation, name="add1")
# register error handler
wg.error_handlers = [handle_negative_sum]

#------------------------- Submit the calculation -------------------
wg.submit(inputs={"add1": {"code": code,
                            "x": orm.Int(1),
                           "y": orm.Int(-2)
                           },
                },
          wait=True)
print("Task finished OK? ", wg.tasks["add1"].process.is_finished_ok)
print("Exit code: ", wg.tasks["add1"].process.exit_code)
print("Exit Message: ", wg.tasks["add1"].process.exit_message)


WorkGraph process created, PK: 77338
Task finished OK?  True
Exit code:  None
Exit Message:  None


We can confirm that the task first fails again with a 410. Then the WorkGraph restarts the task with the new inputs, and it finishes successfully. 

In [12]:
%verdi process status 77338

WorkGraph<normal_graph><77338> Finished [0]
    ├── ArithmeticAddCalculation<77339> Finished [410]
    └── ArithmeticAddCalculation<77345> Finished [0]


## Compare to the `BaseRestartWorkChain`
AiiDA provides a `BaseRestartWorkChain` class that can be used to write workflows that can handle known failure modes of processes and calculations.

In [19]:
from aiida.engine import BaseRestartWorkChain
from aiida.plugins import CalculationFactory
from aiida import orm
from aiida.engine import while_
from aiida.engine import process_handler, ProcessHandlerReport

ArithmeticAddCalculation = CalculationFactory('core.arithmetic.add')

class ArithmeticAddBaseWorkChain(BaseRestartWorkChain):

    _process_class = ArithmeticAddCalculation


    @classmethod
    def define(cls, spec):
        """Define the process specification."""
        super().define(spec)
        spec.expose_inputs(ArithmeticAddCalculation, namespace='add')
        spec.expose_outputs(ArithmeticAddCalculation)
        spec.outline(
            cls.setup,
            while_(cls.should_run_process)(
                cls.run_process,
                cls.inspect_process,
            ),
            cls.results,
        )

    def setup(self):
        """Call the `setup` of the `BaseRestartWorkChain` and then create the inputs dictionary in `self.ctx.inputs`.

        This `self.ctx.inputs` dictionary will be used by the `BaseRestartWorkChain` to submit the process in the
        internal loop.
        """
        super().setup()
        self.ctx.inputs = self.exposed_inputs(ArithmeticAddCalculation, 'add')
    
    @process_handler
    def handle_negative_sum(self, node):
        """Check if the calculation failed with `ERROR_NEGATIVE_NUMBER`.

        If this is the case, simply make the inputs positive by taking the absolute value.

        :param node: the node of the subprocess that was ran in the current iteration.
        :return: optional :class:`~aiida.engine.processes.workchains.utils.ProcessHandlerReport` instance to signal
            that a problem was detected and potentially handled.
        """
        if node.exit_status == ArithmeticAddCalculation.exit_codes.ERROR_NEGATIVE_NUMBER.status:
            self.ctx.inputs['x'] = orm.Int(abs(node.inputs.x.value))
            self.ctx.inputs['y'] = orm.Int(abs(node.inputs.y.value))
            return ProcessHandlerReport()


In the `BaseRestartWorkChain`, the error handling is implemented for a specific Calculation class. While, the error handling in a WorkGraph is more general and can be applied to any task in the WorkGraph.

## Summary
Here we have shown how to implement error handling in a WorkGraph.